In [1]:
import gym
import sys
import multiprocessing
import numpy as np
import tensorflow as tf
from baselines.ppo2 import ppo2
from baselines.ppo2.policies import CnnPolicy
from baselines.common.vec_env.vec_frame_stack import VecFrameStack
from baselines.common.cmd_util import make_atari_env

In [2]:
load_path = '/tmp/openai-2018-05-18-11-23-51-343727/checkpoints/01600'

In [2]:
env = VecFrameStack(make_atari_env('BreakoutNoFrameskip-v4',8,0),4)

In [4]:
obs = env.reset()
obs.shape
env.observation_space

Box(84, 84, 4)

In [5]:
ncpu = multiprocessing.cpu_count()
if sys.platform == 'darwin': ncpu //= 2
config = tf.ConfigProto(allow_soft_placement=True,
                        intra_op_parallelism_threads=ncpu,
                        inter_op_parallelism_threads=ncpu)
config.gpu_options.allow_growth = True #pylint: disable=E1101
tf.Session(config=config).__enter__()

In [6]:
policy = CnnPolicy
ob_space = env.observation_space
ac_space = env.action_space
nbatch_act = 1 * 8
nbatch_train = 32 * 8
nsteps = 128
ent_coef = 0.1
vf_coef = 0.5
max_grad_norm = 0.5
make_model = lambda : ppo2.Model(policy=policy, ob_space=ob_space, ac_space=ac_space, 
                            nbatch_act=nbatch_act, nbatch_train=nbatch_train, nsteps=nsteps, 
                            ent_coef=ent_coef, vf_coef=vf_coef, max_grad_norm=max_grad_norm)
model = make_model()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
model.load(load_path)

In [8]:
actions, _, _, _, = model.step(obs)

In [9]:
actions

array([0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
env.step(actions)

(array([[[[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
 
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
 
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
 
         ...,
 
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
 
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
 
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]],
 
 
        [

In [9]:
env.venv.render()

In [12]:
#obs = env.reset()
while True:
    actions, _, _, _, = model.step(obs)
    env.step(actions)
    env.venv.render()
    

KeyboardInterrupt: 

In [2]:
from baselines.ppo2.policies import nature_cnn
from baselines.a2c.utils import conv, fc

In [8]:
obs.shape

(8, 84, 84, 4)

In [3]:
ob_shape=(128,84,84,4)

In [4]:
X = tf.placeholder(tf.uint8, ob_shape)
h = nature_cnn(X)
v_int = fc(h,'v',1)
v_int

<tf.Tensor 'v/add:0' shape=(128, 1) dtype=float32>

In [5]:
v_int[:,0]

<tf.Tensor 'strided_slice:0' shape=(128,) dtype=float32>

In [6]:
pdparam = fc(h, 'pi', 4, init_scale=1.0, init_bias=0.0)
pdparam

<tf.Tensor 'pi/add:0' shape=(128, 4) dtype=float32>

In [9]:
u = tf.random_uniform(tf.shape(pdparam))
u

<tf.Tensor 'random_uniform:0' shape=(128, 4) dtype=float32>

In [5]:
h1 = tf.reshape(h, [2,128,-1])
h1

<tf.Tensor 'Reshape_1:0' shape=(2, 128, 256) dtype=float32>

In [9]:
xs = [tf.squeeze(v, [1]) for v in tf.split(axis=1, num_or_size_splits=128, value=h1)]
xs

[<tf.Tensor 'Squeeze_256:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_257:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_258:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_259:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_260:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_261:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_262:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_263:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_264:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_265:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_266:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_267:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_268:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_269:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_270:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_271:0' shape=(2, 256) dtype=float32>,
 <tf.Tensor 'Squeeze_272:0' shape=(2, 256) dtype=float32

In [7]:
M = tf.placeholder(tf.float32, [256])
M

<tf.Tensor 'Placeholder_1:0' shape=(256,) dtype=float32>

In [10]:
ms = [tf.squeeze(v, [1]) for v in tf.split(axis=1, num_or_size_splits=128, 
                                           value=tf.reshape(M, [2,128,-1]))]
ms

[<tf.Tensor 'Squeeze_384:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_385:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_386:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_387:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_388:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_389:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_390:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_391:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_392:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_393:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_394:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_395:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_396:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_397:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_398:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_399:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_400:0' shape=(2, 1) dtype=float32>,
 <tf.Tensor 'Squeeze_401:0' sha

In [11]:
S = tf.placeholder(tf.float32, [2, 512])
S

<tf.Tensor 'Placeholder_2:0' shape=(2, 512) dtype=float32>

In [19]:
c, h = tf.split(axis=1, num_or_size_splits=2, value=S)
h

<tf.Tensor 'split_8:1' shape=(2, 256) dtype=float32>

In [24]:
xs[0].get_shape()

TensorShape([Dimension(2), Dimension(256)])

In [23]:
m = ms[0]
x = xs[0]
(1-m)

<tf.Tensor 'sub_1:0' shape=(2, 1) dtype=float32>

In [26]:
wx = tf.get_variable('wx', [256, 1024])
wx

<tf.Variable 'wx:0' shape=(256, 1024) dtype=float32_ref>

In [28]:
tf.matmul(x, wx) + tf.matmul(x, wx)

<tf.Tensor 'add:0' shape=(2, 1024) dtype=float32>